In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

In [ ]:
def readFile(path):
    return np.loadtxt(path, dtype=float, skiprows = 1)

def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def readFileFromDialog():
    return readFile(getFilePathFromDialog(dataPath))

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

In [ ]:
def readParticleConfig(path,ConfigurationNumber):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = 1+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    NumberOfAParticles = int(FirstRow.split()[5].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[7].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            BIndex += 1
    return APositions, BPositions

def readParticleConfigFromDialog(ConfigurationNumber):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber)

In [ ]:
def plotMultipleDensityAndPotEnergySeries(Directory,plotDiagrams=False):
    Temperature = re.search(r'(?<=T=).*',Directory).group()
    AllDensitySeries = sorted(glob.glob(Directory+"/p=*"))
    NumberOfSeries = len(AllDensitySeries)
    if plotDiagrams:
        Fig,Ax = plt.subplots(NumberOfSeries,4,squeeze=False, figsize=(17,5*NumberOfSeries))
    PressureList = []
    DensityList = []
    for i in range(NumberOfSeries):
        Pressure = re.search(r'(?<=p=).*',AllDensitySeries[i]).group()
        PressureList.append(Pressure)
        DensitySeries = readFile(glob.glob(AllDensitySeries[i]+"/density*")[0])
        PotSeries = readFile(glob.glob(AllDensitySeries[i]+"/PotEnergySeries*")[0])
        PotEnergyAverage = np.mean(PotSeries)
        DensityAverage = np.mean(DensitySeries)
        DensityList.append(DensityAverage)
        if plotDiagrams:
            Ax[i,0].plot(DensitySeries,'.')
            Ax[i,0].set_title("p= "+str(Pressure))
            DensityHistValues, BinEdges, Patches = Ax[i,1].hist(DensitySeries,bins = 20, density = True)
            Ax[i,1].vlines(DensityAverage,0,max(DensityHistValues),color='g',label="DensityAvg={}".format(round(DensityAverage,2)))
            Ax[i,1].set_xlabel('density')
            Ax[i,1].set_ylabel('number of occurences')
            Ax[i,2].plot(PotSeries,'.')
            PotEnergyHistValues, BinEdges, Patches = Ax[i,3].hist(PotSeries,bins = 20, density = True)
            Ax[i,3].vlines(PotEnergyAverage,0,max(PotEnergyHistValues),color='g',label="PotEnergyAvg={}".format(round(PotEnergyAverage,2)))
            Ax[i,3].legend()
    return Temperature,PressureList,DensityList

In [ ]:
def plotPressureDensityDiagramForMultipleTemperatures():
    Directory = getDirectoryPathFromDialog(dataPath)
    Fig,Ax = plt.subplots(1,1,squeeze=False,figsize=(7,7),dpi=200)
    Ax[0,0].set_xlim(0.6,0.91)
    Ax[0,0].set_ylim(1.0,3.8)
    Ax[0,0].set_ylabel('pressure')
    Ax[0,0].set_xlabel('density')
    Ax[0,0].set_title('p_vs_roh_finer_scan')
    for Directory in sorted(glob.glob(Directory+'/T=*')):
        Temperature, PressureList, DensityList = plotMultipleDensityAndPotEnergySeries(Directory,False)
        Ax[0,0].plot(np.array(DensityList),np.array(PressureList,dtype=float),'.-',label=('T='+str(Temperature)))
    Ax[0,0].legend(bbox_to_anchor=(1,1), loc="upper left")
    #Fig.savefig('plots/p_vs_roh_finer_scan.png',format='png',dpi=200)

In [ ]:
dataPath = getDirectoryPathFromDialog('')

## Plot single temperature p vs roh diagram

In [ ]:
Temperature, PressureList, DensityList = plotMultipleDensityAndPotEnergySeries(getDirectoryPathFromDialog(dataPath),True)

In [ ]:
Fig,Ax = plt.subplots(1,1,squeeze=False,figsize=(5,5))
Ax[0,0].plot(np.array(DensityList),np.array(PressureList,dtype=float),'.')
#Ax[0,0].set_yscale('log')
Ax[0,0].set_ylabel('pressure')
Ax[0,0].set_xlabel('density')
Ax[0,0].set_title('T='+str(Temperature)+'_p_vs_roh_finer_scan')
Ax[0,0].set_xlim(0.7,0.91)
#Ax[0,0].set_ylim(0.5,4.0)
Fig.savefig('plots/T='+str(Temperature)+'_p_vs_roh_finer_scan.png',format='png',dpi=200)

## Plot multiple temperature p vs roh diagram

In [ ]:
plotPressureDensityDiagramForMultipleTemperatures()

## Plot single density series

In [ ]:
DensitySeries = readFileFromDialog()

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(5,5))
Ax[0,0].plot(DensitySeries)

## Plot single pot energy series

In [ ]:
PotEnergySeries = readFileFromDialog()

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(5,5))
Ax[0,0].plot(PotEnergySeries)

## Plot particle position

In [ ]:
APositions, BPositions = readParticleConfigFromDialog(0)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =100)
Host[0,0].plot(APositions[:,0],APositions[:,1],'.',color='b',ms=2)
Host[0,0].plot(BPositions[:,0],BPositions[:,1],'.',color='darkorange',ms=2)
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
Host[0,0].set_xlim(0,1)
Host[0,0].set_ylim(0,1)